In [ ]:
%reset -f
import glob
import math
import numpy as np
from numpy.lib.function_base import meshgrid
import pandas as pd

from importlib import reload
#syntax: fugi = reload(fugi)

import functions_giammi as fugi

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
%config InlineBackend.figure_formats=["png"]

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pyntcloud import PyntCloud, structures

plt.style.use(hep.style.ROOT)  # For now ROOT defaults to CMS
# hep.set_style({"font.size":8})
# mpl.rc('font', size=8)

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#!S-TFMOx_oldPhilippTheory
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=3.0eV';en="KE=3eV";
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=4.0eV';en="KE=4eV";
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=6.1eV';en="KE=6.1eV";
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=8.2eV';en="KE=8.2eV";
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=11.5eV';en="KE=11.5eV";

#S-TFMOx
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S_22/enant=S_hel=+1_KE=3.1eV';en="KE=3.1eV";
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S_22/enant=S_hel=+1_KE=4.1eV';en="KE=4.1eV";
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S_22/enant=S_hel=+1_KE=6.1eV';en="KE=6.1eV";
# path = r'../PYTHON_graphs/DATA/Theory/TFMox-S_22/enant=S_hel=+1_KE=8.1eV';en="KE=8.1eV";
path = r'../PYTHON_graphs/DATA/Theory/TFMox-S_22/enant=S_hel=+1_KE=11.7eV';en="KE=11.7eV";

#MOx
# path = r'../PYTHON_graphs/DATA/Theory/MOx-R/MoX_CPR_positive';en="MOx_CPR";
# path = r'../PYTHON_graphs/DATA/Theory/MOx-R/MoX_CPL_negative';en="MOx_CPL";

# glob has to be sorted (both in unix and windows)
all_files = sorted(glob.glob(path + "/**.dat"))

#if True it computes the 72 MFPAD
enant="R"
MFPAD=False
optimization=True

li = []; cosphi_th = []; test_opening = []; allnames=[]
colnames = ["phi", "cos(theta)", "value"]

#how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/,
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None)  #r"\s+" is a regex (regular expression)
    #or delim_whitespace=True, it is faste
    temp_name=(filename.replace(" ","")).split("/")[-1].split(".")[0] #adding a column to df with the file name with no spaces and extension
    if temp_name.split("_")[0] != 'MFPAD':
        cosphi_th.append((int(temp_name.split("_")[0]), int(temp_name.split("_")[1])))
        allnames.append(temp_name)
        df["filename"]=temp_name
        df["filename"]=df["filename"].astype("category")
        li.append(df) #a unique DataFrame
    else:
        continue

frame = pd.concat(li, axis=0)
#this commmand avoids 1_1 followed by 1_10: semantic order
frame_srt = frame.groupby("filename",sort=False)
frame_set = frame.set_index("filename") #build a multiindex using the categories of filename

In [ ]:
#TODO optimize the parameters for the new surfaces (the depth is probably too high)

In [ ]:
"""
##############################################NOTE####################################################
in elements tag:
cosphi_th=([[ 1,  1],   cosphi_th_cos=([[ 6,  1],       cosphi_phi=([[ 6,  1],      
            [ 1,  2],                   [ 6,  2],                    [ 5,  1],                   
            [ 1,  3],                   [ 6,  3],                    [ 4,  1],
            [ 1,  4],                   [ 6,  4],                    [ 3,  1],
            [ 1,  5],                   [ 6,  5],                    [ 2,  1],                   
            [ 1,  6],                   [ 6,  6],                    [ 1,  1],

in angles tag:
cosphi_adj_th=([[ 1,  -180],   cosphi_adj_cos=([[ -1,  -180],       cosphi_adj_phi=([[ -1,   -180],      
                [ 1,  -147],                    [ -1,  -147],                        [ -0.8, -180],                   
                [ 1,  -114],                    [ -1,  -114],                        [ -0.3, -180],
                [ 1,  -81],                     [ -1,  -81],                         [ 0.3,  -180],
                [ 1,  -49],                     [ -1,  -49],                         [ 0.8,  -180],                   
                [ 1,  -16],                     [ -1,  -16],                         [ 1,    -180],

cosphi_th has to be used JUST with counts_stemp!!

b1 maps cordinates. YOU SHOULD USE THIS AS ROTATION INPUT

list(zip(xgo_phi,ygo_phi))             list(zip(xgo_cos,ygo_cos))
[(-180.0, 1.0),                        [(-180.0, 1.0),
 (-180.0, 0.809),                       (-147.273, 1.0),
 (-180.0, 0.309),                       (-114.545, 1.0),
 (-180.0, -0.309),                      (-81.818, 1.0),
 (-180.0, -0.809),                      (-49.091, 1.0),
 (-180.0, -1.0),                        (-16.364, 1.0),
##############################################NOTE####################################################
######################################################################################################
                                    PHI - CTHETEA ELECTRON 
######################################################################################################
(alpha and beta in Philipp convention)
REMEBER: in Philipp simulations cos(theta)=[1,-1] becase theta=[0,np.pi]
SORTED = cos(theta)=[-1,1] IS NEVER USED for electrons coordinates
#NOTE: the following phi and ctheta are not unique: len=20000

"""
#SORTED FULL **cos(theta)_el = [1,-1]**, phi_el = [-179.1, 179.1]
# len = 200000
phi_full_sorted=frame_srt.get_group("1_1").sort_values(by=["phi","cos(theta)"])["phi"] 
phi_rad_sorted = phi_full_sorted * np.pi/180.

ctheta_full_sorted=frame_srt.get_group("1_1").sort_values(by=["phi","cos(theta)"])["cos(theta)"]
theta_rad_sorted = np.arccos(ctheta_full_sorted)
theta_full = theta_rad_sorted*180./np.pi

#SORTED unique **cos(theta)_el = [1,-1]**, phi_el = [-179.1, 179.1]  
# len(phiMs) = 200, len(cosM) = 100
phiM=frame_srt.get_group("1_1").sort_values(by=["phi","cos(theta)"])["phi"].unique()
cosM=frame_srt.get_group("1_1").sort_values(by=["phi","cos(theta)"])["cos(theta)"].unique()

#UNSORTED FULL phi_el = [-179.1, 179.1]
# len = 200000
# phi_full = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy()
# phi_rad = phi_full * np.pi/180.

#UNSORTED FULL cos(theta)_el = [-1,1]
# len = 200000
# ctheta_full = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy()
# theta_rad = np.arccos(ctheta_full)
# theta_full = theta_rad*180./np.pi

#UNSORTED unique cos(theta)_el = [-1,1], phi_el = [-179.1, 179.1]
# len(phiMu) = 200, len(cosMu) = 100
#NOTE      np.any(phiM == phiMu) -> TRUE
#          np.any(cosM == np.flip(cosMu)) -> TRUE
phiMu = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi_el_u = [-179.1, 179.1]
cosMu = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)_el_u = [-1,1]

#NOTE: for np.MESHGRID:
#for the 2D case with inputs of length M and N, the outputs are of shape (N, M) for ‘xy’ indexing and (M, N) for ‘ij’ indexing. Cartesia "xy" by default
#REMEBER: np.any(phiM == phiMs) -> TRUE

#This is the right one to use to interpolate if the MFPAD IS SORTED
# because cos(theta)_el = [1,-1]
phiMM, cosMM = np.meshgrid(phiM, cosM)
phiMM_rad, thetaMM_rad = np.meshgrid(phiM*np.pi/180., np.arccos(cosM))

#because **cos(theta)_el = [1,-1]**
# phiMMu, cosMMu = np.meshgrid(phiMu, cosMu)
# phiMMu_rad, thetaMMu_rad = np.meshgrid(phiMu*np.pi/180., np.arccos(cosMu))

print("pihM ", phiM.shape, "cosM ", cosM.shape)
print("cosM[0] ", cosM[0])
print("pihMM ", phiMM.shape, "cosMM ", cosMM.shape)
print(type(phiM), type(phiMM))

In [ ]:
"""
##########################b1 map x,y coordiantes###############################
PHI - CTHETA PHOTON with respect to the LF (in sperical coordinates)
NOTE the go coordiantes are organised accordinf to the SORTED MFPAD, therefore cos(theta)_PHOTON=[-1,1]
REDUCED=TRUE => ADJUSTED TO EXPERIMENT: -0.835,0.835,-165,165
REDUCED=FALSE => ADJUSTED TO THEORY: -1,1,-180,180
"""
# fugi = reload(fugi)
#the std goes cos(theta)_photon [1,-1] therefore uses cos_th_adj
#the sorted cos has cos(theta)_photon [1,-1] therefoere uses cos_th_adj_cflip
#deprecated: it is linearly disributed in cos(theta), but the real data are linear in theta!
# cosphi_th_adj=np.around(np.array(list(itertools.product(np.linspace(1.,-1.,6).tolist(),np.linspace(-180,180,12).tolist()))),3)
# cosphi_th_adj_exp = np.around(np.array(list(itertools.product(np.linspace(0.835,-0.835,6).tolist(),np.linspace(-165,165,12).tolist()))),3)
# cosphi_th_adj_exp_cflip = np.around(np.array(list(itertools.product(np.linspace(-0.835,0.835,6).tolist(),np.linspace(-165,165,12).tolist()))),3)

# half of 30 deg in radiants 
# cos_lin=False; limits=[np.pi-0.261799388, 0.261799388, -180., 180.] #linear in theta [np.pi,0]
cos_lin=True; limits=[-1+1/6, 1-1/6, -180+33/2, 180-33/2] #linear in ctheta [-1,1], default with Philipp
# cos_lin=True; limits=[-0.95,0.95, -175., 175.] #linear in ctheta [-1,1], default with Philipp
# cos_lin=True; limits=[-1,1,-180,180] #linear in ctheta reduced experiment

#probably linear in theta just in the theory
xgo_phi,ygo_phi,cosphi_adj_phi = fugi.create_gocoords(a=0, cos_lin=cos_lin, limits=limits, source=True)
xgo_cos,ygo_cos,cosphi_adj_cos = fugi.create_gocoords(a=1, cos_lin=cos_lin, limits=limits, source=True)
xgo_cos_flip,ygo_cos_flip,cosphi_adj_cos_flip = fugi.create_gocoords(a=2, cos_lin=cos_lin, limits=limits, source=True)
xgo_phi_flip,ygo_phi_flip,cosphi_adj_phi_flip = fugi.create_gocoords(a=3, cos_lin=cos_lin, limits=limits, source=True)

# xgo_phi,ygo_phi,cosphi_adj_phi = fugi.create_gocoords(a=0, reduced=False, limits=[0.96,177], source=True)
# xgo_cos,ygo_cos,cosphi_adj_cos = fugi.create_gocoords(a=1, reduced=False, limits=[0.96,177], source=True)
# xgo_cos_flip,ygo_cos_flip,cosphi_adj_cos_flip = fugi.create_gocoords(a=2, reduced=False, limits=[0.96,177], source=True)
# xgo_phi_flip,ygo_phi_flip,cosphi_adj_phi_flip = fugi.create_gocoords(a=3, reduced=False, limits=[0.96,177], source=True)

#NOTE to preserve the scanning along cos(theta)_phot or phi_phot meshgrid is needed. unique and zip do not work!
xgo_uniq=np.unique(np.array(xgo_cos));ygo_uniq=np.unique(np.array(ygo_cos))
xgo_flip_uniq=np.unique(np.array(xgo_cos_flip));ygo_flip_uniq=np.unique(np.array(ygo_cos_flip))

# list(zip(xgo_phi,ygo_phi))
"""
Creation of DataFrame to find the right element according to rotation
isuefull fot the 3d representation
NOTE allnames are UNSORTED: provide cosphi_df with cos(thea) = [1,-1] sorted by cos(theta)
NOTE all the coordinates are FULL!! this is iportant for rotation

according to automatic_72_CPR.f90 from philipp [1,6]: is from [0,P1] -> [cos(0), cos(PHI)] = [1,-1]
according to automatic_72_CPR.f90 from philipp [1,12]: is from [-PI,PI]

allnames  cos(theta)   phi
1_1       1            -180 
1_2       1            -147 
"""
#NOTE cosphi_th_adj comes from manual creation cos [1:-1], phi[-180;180], but same order as original
dfind=pd.DataFrame(cosphi_adj_cos_flip, columns=["ctheta","phi"])
dfind["item"]=allnames

#Sorting the label of the element according to the numerical value of ctheta and phi
cosphi_th_std=(dfind.sort_values(by=["ctheta", "phi"], ascending=[False,True]))["item"].values
cosphi_th_cos=(dfind.sort_values(by=["ctheta", "phi"]))["item"].values
cosphi_th_phi_cflip=(dfind.sort_values(by=["phi","ctheta"],ascending=[True,False]))["item"].values
cosphi_th_phi=(dfind.sort_values(by=["phi","ctheta"]))["item"].values


print("(xgo (phi), ygo (ctheta)) \n", list(zip(xgo_cos,ygo_cos))[:6])
print("(xgo flip (phi), ygo flip (ctheta)) \n", list(zip(xgo_cos_flip,ygo_cos_flip))[:6])

print("\n Default from the theory is 1.")
print("\n 1. cosphi_adj_cos_flip  \n", cosphi_adj_cos_flip[:6], "\n cospi_th_std \n", cosphi_th_std[:6], "\n")
print("\n 2. cosphi_adj_cos \n", cosphi_adj_cos[:6], "\n cospi_th_cos \n", cosphi_th_cos[:6], "\n")
print("\n 3. cosphi_adj_phi \n", cosphi_adj_phi[:6], "\n cospi_th_phi \n", cosphi_th_phi[:6], "\n")
print("\n 4. cosphi_adj_phi_flip \n", cosphi_adj_phi_flip[:6], "\n cospi_th_phi_flip \n", cosphi_th_phi_cflip[:6], "\n")

In [ ]:
# convert the counts into a 2D np array and sort according to cos and phi
#NOTE: theta is along the z_MF and phi is atan(x_MF,y_MF)
# fugi = reload(fugi)
counts_temp=[] #standard cos sorted S enantiomer
check_file=[] #standard cos sorted S enantiomer
MFPAD_R_std=[]
MFPAD_R_phi=[]

#NOTE the values of each MFPAD is espressend with phi[-180,180], cos(theta)[1,-1] for ELECTRONS
#counts_temp: in photon_coordiantes cos(theta)=[-1,1], phi=[-180,180] 1_1, 1_2, 1_3 ...
#             in electron coordinates cos(theta)_el=[1,-1], phi_el=[-180,180]
for group in frame_srt:
    counts_temp.append(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["value"].values)
    check_file.append(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["filename"])
    if group[0] == "1_1":
        phi_full_sorted_check=np.array(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["phi"].values)
        ctheta_full_sorted_check=np.array(frame_srt.get_group(group[0]).sort_values(by=["phi","cos(theta)"])["cos(theta)"].values)

#counts_temp: in photon_coordinates as cosphi_th_std: 
# 1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7', '1_8' 
MFPAD_S_std=np.array(counts_temp).reshape(72,200,100)

#MFPAD_S_cos: in photon_coordinates as cosphi_th_std 
# OUTPUT a=1 : 6_1', '6_2', '6_3', '6_4', '6_5', '6_6', '6_7', '6_8'
MFPAD_S_cos=fugi.sorting_array(MFPAD_S_std, theory=True, items=cosphi_th_std, a=1)

#MFPAD_S_phi: in photon_coordiantes as cosphi_th_std 
# OUTPUT a=2 : 6_1', '5_1', '4_1', '3_1', '2_1', '1_1', '6_2', '5_2'
MFPAD_S_phi=fugi.sorting_array(MFPAD_S_std, theory=True, items=cosphi_th_std, a=2)

print("order: phi == phi_check ",np.all(phi_full_sorted == phi_full_sorted_check))
print("order: phi == phi_check ",np.all(ctheta_full_sorted == ctheta_full_sorted_check))
#these are all false because of the cos(theta)_photon invertion
print("order: original == phi_ph ",np.all(counts_temp == MFPAD_S_phi))
print("order: original == cos_ph ",np.all(counts_temp == MFPAD_S_cos))

"""
Whith this sorting:
MFPAD_S_phi == MFPAD_S_cos gives True for the first and the last element
MFPAD_S_phi or MFPAD_S_phi == counts_temp are False, because the cos(theta) PHOTON is changed to [-1,1] from the original [1,-1]
"""
#flipping for R enantiomer and sorting: JUST ALONG COS(theta) -> axis=1
for el in MFPAD_S_std:
    MFPAD_R_std.append(np.flip(el,axis=1))
    # MFPAD_R_cos.append(np.flip(MFPAD_R_cos, axis=1))

for el in MFPAD_S_phi:
    MFPAD_R_phi.append(np.flip(el,axis=1))
    # MFPAD_R_cos.append(np.flip(MFPAD_R_cos, axis=1))

#Sum in indepedent of the sorting, it is a total sum:
# MFPAD_R_phi_sum is useless!
# np.any(MFPAD_S_cos_sum == MFPAD_S_phi_sum) TRUE
MFPAD_S_phi_sum=np.sum(MFPAD_S_phi,axis=0)
MFPAD_R_phi_sum=np.sum(MFPAD_R_phi,axis=0)
# MFPAD_R_phi_sum=np.sum(MFPAD_R_phi,axis=0)

MFPAD_S_phi_sum_norm = fugi.normalise_with_err(MFPAD_S_phi_sum,normtype=3)
MFPAD_R_phi_sum_norm = fugi.normalise_with_err(MFPAD_R_phi_sum,normtype=3)

print(np.array(MFPAD_S_phi).shape)
print(np.array(MFPAD_R_phi).shape)

In [ ]:
if optimization:
    #EULER angles for optimized rotation from distance optimization anglorithm 
    CH = "CH11"
    if en == "KE=3.1eV":
        a=12.;b=13.;c=-9.;#550
    if en == "KE=4.1eV":
        a=2.;b=10.;c=-13.; #546
    if en == "KE=6.1eV":
        a=23.;b=7.;c=-10.; #544
    if en == "KE=8.1eV":
        a=7.;b=1.;c=-11.; #542
    if en == "KE=11.7eV":
        a=6.;b=5.;c=-14.; #541
    print("a= ", a,"; b= ", b,"; c= ", c)
    
    cosphi_adj_phi_rot=np.column_stack((np.ones(72)*np.cos(b*np.pi/180.),np.ones(72)*a,np.ones(72)*c))
    
    MFPAD_S_phi_rot_LF,dummy1,dummy2=fugi.rot3d_MFPAD(MFPAD_S_phi,theta_rad_sorted,phi_rad_sorted,cosphi_adj_phi_rot,phiM,cosM,convention=1,s=0.0001)
    MFPAD_S_phi_rot_LF_norm = fugi.normalise_with_err(MFPAD_S_phi_rot_LF,normtype=3)
    temp=[];temp1=[]
    for el in MFPAD_S_phi_rot_LF:
        temp.append(np.flip(el, axis=1))
    for el in MFPAD_S_phi_rot_LF_norm:
        temp1.append(np.flip(el, axis=1))
    MFPAD_R_phi_rot_LF=np.array(temp)
    MFPAD_R_phi_rot_LF_norm=np.array(temp1)
    # MFPAD_R_phi_rot_LF,dummy1,dummy2=fugi.rot3d_MFPAD(MFPAD_R_phi,theta_rad_sorted,phi_rad_sorted,cosphi_adj_phi_rot,phiM,cosM,convention=1,s=0.0001)
    # MFPAD_R_phi_rot_LF_norm = fugi.normalise_with_err(MFPAD_R_phi_rot_LF,normtype=3)

    MFPAD_S_phi_rot_LF_sum=np.nan_to_num(np.sum(MFPAD_S_phi_rot_LF,axis=0))
    MFPAD_R_phi_rot_LF_sum=np.nan_to_num(np.sum(MFPAD_R_phi_rot_LF,axis=0))


In [ ]:
"""
Creation of DataFrame to find the right element according to rotation
isuefull fot the 3d representation
NOTE allnames are UNSORTED: provide cosphi_df with cos(thea) = [1,-1] sorted by cos(theta)
NOTE all the coordinates are FULL!! this is iportant for rotation

according to automatic_72_CPR.f90 from philipp [1,6]: is from [0,P1] -> [cos(0), cos(PHI)] = [1,-1]
according to automatic_72_CPR.f90 from philipp [1,12]: is from [-PI,PI]

allnames  cos(theta)   phi
1_1       1            -180 
1_2       1            -147 
"""
#NOTE cosphi_adj_cos (or phi?) comes from create_gocoords, therefore can be reduced to meet the experiments, or not
dfind=pd.DataFrame(cosphi_adj_phi, columns=["ctheta","phi"])
dfind["item"]=allnames

#Sorting the label of the element according to the numerical value of ctheta and phi
#NOTE: BOTH will have cos(theta) = [-1,1], regardless from the sorting
cosphi_th_cos=(dfind.sort_values(by=["ctheta", "phi"]))["item"].values
cosphi_th_phi=(dfind.sort_values(by=["phi","ctheta"]))["item"].values

In [ ]:
# find the photon rotation vakues of a specific set of rotation
# where b1 is ma or min!
# dfind.sort_values(by=["phi", "ctheta"])
dfind.sort_values(by=["ctheta","phi"])
dfind.loc[dfind["item"] == "4_7"]
dfind.loc[dfind["phi"] == 74.318]

#selecting the rght file according to the needs ["filename"] == "x_x"]
nitem=37
# nitem=4

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(25, 10))
aspect=(phiM.max()/cosM.max())

title_fig=enant+" MFPAD el "+str(nitem)+" "+en
if enant == "R":
    cs0,ax[0]=fugi.plot_interpolation(phiM,cosM,MFPAD_R_phi[nitem].T,ax=ax[0], ystep=0.01, n=20)
else:
    cs0,ax[0]=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi[nitem].T,ax=ax[0], ystep=0.01, n=20)
ax[0].set_title(title_fig)
ax[0].set_ylabel('cos\u03D1 [adm]')
ax[0].set_xlabel('\u03C6 [DEG]')
ax[0].set_aspect(aspect)
if optimization:
    title_fig=title_fig+"_OPT"
    if enant == "R":
        cs1,ax[1]=fugi.plot_interpolation(phiM,cosM,MFPAD_R_phi_rot_LF[nitem].T,ax=ax[1], ystep=0.01, n=20)
    else:
        cs1,ax[1]=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi_rot_LF[nitem].T,ax=ax[1], ystep=0.01, n=20)
    ax[1].set_title(title_fig)
    ax[1].set_ylabel('cos\u03D1 [adm]')
    ax[1].set_xlabel('\u03C6 [DEG]')
    ax[1].set_aspect(aspect)
    

fig.savefig("OUTPUTS/Theory/MFPAD_el_"+enant+"_"+str(nitem)+"_"+en+"_theory.png")

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(25, 10))
aspect=(phiM.max()/cosM.max())

title_fig=enant+" MFPAD sum "+en
if enant == "R":
    cs0,ax[0]=fugi.plot_interpolation(phiM,cosM,MFPAD_R_phi_sum.T,ax=ax[0], ystep=0.01, n=20)
else:
    cs0,ax[0]=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi_sum.T,ax=ax[0], ystep=0.01, n=20)
ax[0].set_title(title_fig)
ax[0].set_ylabel('cos\u03D1 [adm]')
ax[0].set_xlabel('\u03C6 [DEG]')
ax[0].set_aspect(aspect)
if optimization:
    title_fig=title_fig+"_OPT"
    if enant == "R":
        cs1,ax[1]=fugi.plot_interpolation(phiM,cosM,MFPAD_R_phi_rot_LF_sum.T,ax=ax[1], ystep=0.01, n=20)
    else:
        cs1,ax[1]=fugi.plot_interpolation(phiM,cosM,MFPAD_S_phi_rot_LF_sum.T,ax=ax[1], ystep=0.01, n=20)
    ax[1].set_title(title_fig)
    ax[1].set_ylabel('cos\u03D1 [adm]')
    ax[1].set_xlabel('\u03C6 [DEG]')
    ax[1].set_aspect(aspect)
    

fig.savefig("OUTPUTS/Theory/MFPAD_"+enant+"_el_sum_"+en+"_theory.png")

In [ ]:
# convertion to spherical coordinates 1D vectors of shape (20000,1)
if optimization:
    if enant == "R":
        counts = MFPAD_R_phi_rot_LF[nitem].reshape(-1)
        countss = MFPAD_R_phi_rot_LF_sum.reshape(-1)    
    else:
        counts = MFPAD_S_phi_rot_LF[nitem].reshape(-1)
        countss = MFPAD_S_phi_rot_LF_sum.reshape(-1)    
else:
    if enant == "R":
        counts = MFPAD_R_phi[nitem].reshape(-1)
        countss = MFPAD_R_phi_sum_norm.reshape(-1)    
    else:
        counts = MFPAD_S_phi[nitem].reshape(-1)
        countss = MFPAD_S_phi_sum_norm.reshape(-1)

#single MFPAD
x = counts * np.sin(theta_rad_sorted) * np.cos(phi_rad_sorted)
X = np.array(x).reshape(200,100)
y = counts * np.sin(theta_rad_sorted) * np.sin(phi_rad_sorted)
Y = np.array(y).reshape(200,100)
z = counts * np.cos(theta_rad_sorted)
Z = np.array(z).reshape(200,100)

d = np.sqrt(x**2+y**2+z**2)
d_matrix = np.sqrt(X**2+Y**2+Z**2)

#sum MFPAD
xs = countss * np.sin(theta_rad_sorted) * np.cos(phi_rad_sorted)
Xs = np.array(xs).reshape(200,100)
ys = countss * np.sin(theta_rad_sorted) * np.sin(phi_rad_sorted)
Ys = np.array(ys).reshape(200,100)
zs = countss * np.cos(theta_rad_sorted)
Zs = np.array(zs).reshape(200,100)

ds = np.sqrt(xs**2+ys**2+zs**2)
d_matrixs = np.sqrt(Xs**2+Ys**2+Zs**2)


fig=plt.figure()
fig.show()
ax=fig.add_subplot(111)

#ax.plot(counts_fixed[0:200]-counts[0:200],c="b",label='diff_counts[0:200]')
ax.plot(counts_fixed-counts,c="b",label='diff_counts')
ax.plot(a1-b1,c="g",label='diff_+phi_-phi')
ax.plot(a2-b2,c="y",label='diff_+ctheta_-ctheta')
#ax.plot(avgcos[0:200]-counts[0:200],c="r",label='diff_+avgcos_-counts[0:200]')
#ax.plot(avgphi[0:200]-counts[0:200],c="m",label='diff_+avgphi_-counts[0:200]')
#ax.set_yscale('log')
plt.legend(loc="best")
print(np.min(phi_fixed),np.max(phi_fixed),np.min(ctheta_fixed),np.max(ctheta_fixed))

In [ ]:
# Go scatters do not provide a legend
if optimization:
  title=enant+"-TFMOx OPT MFPAD theory scattered"
else:
  title=enant+"-TFMOx MFPAD theory scattered"

fig = go.Figure(data=[go.Scatter3d(x=xs, y=ys, z=zs,
                    mode='markers',
                    marker=dict(
                        size=5,
                        color=ds,            # set color to an array/list of desired values
                        colorscale='Viridis',   # choose a colorscale
                        opacity=1.,
                        showscale=True          # to show the legend according to the color
                   )
                )])

fig.update_layout(title=title,
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                #   margin=dict(l=0, r=0, b=0, t=0))
fig.show()


In [ ]:
if optimization:
  title=enant+"-TFMOx OPT MFPAD theory sur NO FIX "+str(nitem)
else:
  title=enant+"-TFMOx MFPAD theory sur NO FIX "+str(nitem)

fig = go.Figure(data=[go.Surface(z=Zs[0:200,0:100], x=Xs[0:200,0:100], y=Ys[0:200,0:100], surfacecolor=d_matrixs, colorscale='Viridis', connectgaps=True)])
fig.update_layout(title=title, autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()

the Z distribution has the ending and the starting data that aren't matching. This is regardless the reshaping

In [ ]:
if optimization:
  title=enant+"-TFMOx OPT MFPAD theory sur NO FIX "+str(nitem)
  save_path="../PYTHON_graphs/OUTPUTS/3D_cartesian_OPT_"+str(nitem)+"_"+en+"_theory.html"
else:
  title=enant+"-TFMOx MFPAD theory sur NO FIX "+str(nitem)
  save_path="../PYTHON_graphs/OUTPUTS/3D_cartesian_"+str(nitem)+"_"+en+"_theory.html"

fig = go.Figure(data=[go.Surface(z=Z[0:200,0:100], x=X[0:200,0:100], y=Y[0:200,0:100], surfacecolor=d_matrix, colorscale='Viridis', connectgaps=True)])
fig.update_layout(title=title, autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()

fig.write_html(save_path)

It is possible to force the equality and therefore bridging the gap

In [ ]:
#fixing the stripe single
Y[-1,0:100]=Y[0,0:100]
X[-1,0:100]=X[0,0:100]
Z[-1,0:100]=Z[0,0:100]

#fixing the stripe sum
Ys[-1,0:100]=Ys[0,0:100]
Xs[-1,0:100]=Xs[0,0:100]
Zs[-1,0:100]=Zs[0,0:100]

#to share single MFPDAD variables to other notebooks
%store X
%store Y
%store Z
%store d_matrix

In [ ]:
#fixed stripe
if optimization:
  title=enant+"-TFMOx OPT MFPAD theory surf "+str(nitem)
else:
  title=enant+"-TFMOx MFPAD theory surf "+str(nitem)
fig = go.Figure(data=[go.Surface(z=Z[0:200,0:100], x=X[0:200,0:100], y=Y[0:200,0:100], surfacecolor=d_matrix, colorscale='Viridis', connectgaps=True)])
fig.update_layout(title=title, autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
if optimization:
  title=enant+"-TFMOx OPT MFPAD theory surf"
else:
  title=enant+"-TFMOx MFPAD theory surf"
fig = go.Figure(data=[go.Surface(z=Z[0:200,0:100], surfacecolor=d_matrix, connectgaps=True)])
fig.update_layout(title=title, autosize=False,
                #   width=500, height=500,
                #   margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
def lod_mesh_export(mesh, lods, extension, path, enant, optimization):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        if optimization:
            if enant == "R":
                o3d.io.write_triangle_mesh(path+"R_lod_rot_"+str(i)+extension, mesh_lod)
            else:
                o3d.io.write_triangle_mesh(path+"S_lod_rot_"+str(i)+extension, mesh_lod) #for 11.5 eV
        else:
            if enant == "R":
                o3d.io.write_triangle_mesh(path+"R_lod_"+str(i)+extension, mesh_lod)
            else:
                o3d.io.write_triangle_mesh(path+"S_lod_"+str(i)+extension, mesh_lod) #for 11.5 eV
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods

In [ ]:
import open3d as o3d
import sys
import trimesh

sys.path.append('..'); output_path=(r"../PYTHON_graphs/OUTPUTS/meshes/"+en+"/")
alpha_mesh=[]; poisson_mesh=[]; densities=[]

alpha_mesh = o3d.geometry.TriangleMesh()
pcd = o3d.geometry.PointCloud()
pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3)))  # invalidate existing normals
camera_loc=np.array([0.,0.,-1.])

#load the point cloud: select the desired point cloud
# point_cloud=np.array([xs,ys,zs]).T #single MFPAD -> chanhe the number in the outputfile (e.g. 37 or 4)
point_cloud=np.array([x,y,z]).T #sum MFPAD

cloud = PyntCloud.from_instance("open3d", pcd)
pcd.points = o3d.utility.Vector3dVector(point_cloud)
#resise the scale of the sample
vox_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, 1.)

#KD neighbour
kdtree = cloud.add_structure("kdtree")
# dela3d = cloud.add_structure("delaunay3D")
testc = cloud.get_neighbors(k=10)
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)

#compute the normals
pcd.estimate_normals(); #mandatory
# mesh.compute_vertex_normals()
# pcd.normals = o3d.utility.Vector3dVector(point_cloud) #perfectly radial and long, independent by the tangent plane
# They seem good for open surfaces, but not for my case
# pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05,max_nn=20));
# pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50));

'''orient the normals'''
#Number of nearest neighbours: 5 is the minimum to have a closed surface with scale >= 2
pcd.orient_normals_consistent_tangent_plane(7) #higher #bins
# pcd.orient_normals_consistent_tangent_plane(5) #normal bins
# pcd.orient_normals_towards_camera_location(camera_loc) #normal bins
# pcd.orient_normals_to_align_with_direction([0.,1.,0.]) #normal bins

'''####################################################################
#########################alpha-shape algorithm#########################
####################################################################'''
# alpha=np.log10(0.5)
# alpha=0.1
# tetralpha_mesh, pt_map = o3d.geometry.TetraMesh.create_from_point_cloud(pcd)
# alpha_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha, tetralpha_mesh, pt_map) #it resturns zero for some reason

'''###############################################################
###################### Poisson algorithm #########################
###############################################################'''
#high bins
# poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10, width=0, scale=2.5, linear_fit=False)
# poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9, width=0, scale=2., linear_fit=False)
#normal bins single MFAD
poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=7, width=0, scale=1.2, linear_fit=False)
#normal bins tot MFPAD
# poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10, width=0, scale=1.2, linear_fit=False)

bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)
# p_mesh_crop.compute_triangle_normals() # usually computed before rendering (??)

'''################################################################
####################### Hull ball algoritm ########################
###############################################################'''
# radii = [0.08, 0.1, 0.2, 0.4] #multiple radii!
# bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector(radii))

#both for single and tot MFPAD
radius = 1.5 * avg_dist
# radius = 3.5 * avg_dist
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))

'''
CLEANING THE TRIANGLES
arget_number_of_triangles (int) – The number of triangles that the simplified mesh should have. It is not guaranteed that this number will be reached.
maximum_error (float, optional, default=inf) – The maximum error where a vertex is allowed to be merged
boundary_weight (float, optional, default=1.0) – A weight applied to edge vertices used to preserve boundaries
'''
dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)
dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

#none of the followings is working with poisson
# p_mesh_crop =poisson_mesh.simplify_quadric_decimation(6000)
# p_mesh_crop.remove_unreferenced_vertices
# p_mesh_crop.remove_degenerate_triangles()
# p_mesh_crop.remove_duplicated_triangles()
# p_mesh_crop.remove_duplicated_vertices()
# p_mesh_crop.remove_non_manifold_edges()

'''create the triangular mesh with the vertices and faces from open3d to be visualise with another software'''
tri_mesh = trimesh.Trimesh(np.asarray(dec_mesh.vertices), np.asarray(dec_mesh.triangles),vertex_normals=np.asarray(dec_mesh.vertex_normals))
tri_mesh_pois = trimesh.Trimesh(np.asarray(poisson_mesh.vertices), np.asarray(poisson_mesh.triangles),vertex_normals=np.asarray(poisson_mesh.vertex_normals))
# tri_mesh_pois_crop = trimesh.Trimesh(np.asarray(p_mesh_crop.vertices), np.asarray(p_mesh_crop.triangles),vertex_normals=np.asarray(p_mesh_crop.vertex_normals))
# trimesh.convex.is_convex(dec_mesh)

In [ ]:
# np.asarray(poisson_mesh.vertices).T
xgo, ygo, zgo =np.asarray(poisson_mesh.vertices).T
dgo = np.sqrt(xgo**2+ygo**2+zgo**2)

# np.asarray(poisson_mesh.vertex_normals)
# poisson_mesh.triangle_normals
tigo, tjgo, tkgo =np.asarray(poisson_mesh.triangles).T
ti, tj, tk = tri_mesh_pois.triangles.T

# asarray(p_mesh_crop.triangles)
# txgo, tygo, tzgo =np.asarray(p_mesh_crop.vertices).T[0]
# tdgo = np.sqrt(txgo**2+tygo**2+tzgo**2)

In [ ]:
#designing the surface colour
#densities are the real density of features from o3d
densities = np.asarray(densities)
density_colors = plt.get_cmap('viridis')((dgo - dgo.min()) / (dgo.max() - dgo.min()))
density_colors = density_colors[:, :3]

#works for the plotting in o3d
poisson_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
bpa_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
tri_mesh_pois.vertex_colors = o3d.utility.Vector3dVector(density_colors)

if enant == "R":
    o3d.io.write_triangle_mesh(output_path+"R_MFPAD_tot_bpa_mesh_theory.ply", dec_mesh);
    # o3d.io.write_triangle_mesh(output_path+"R_MFPAD_37_bpa_mesh_theory.ply", dec_mesh);
    # o3d.io.write_triangle_mesh(output_path+"R_MFPAD_4_bpa_mesh_theory.ply", dec_mesh);
    o3d.io.write_triangle_mesh(output_path+"R_MFPAD_tot_p_mesh_theory.ply", poisson_mesh);
    # o3d.io.write_triangle_mesh(output_path+"R_MFPAD_37_p_mesh_theory.ply", poisson_mesh);
    # o3d.io.write_triangle_mesh(output_path+"R_MFPAD_4_p_mesh_theory.ply", poisson_mesh);
    # o3d.io.write_triangle_mesh(output_path+"R_MFPAD_p_mesh_c_theory.ply", p_mesh_crop);
else:
    o3d.io.write_triangle_mesh(output_path+"S_MFPAD_tot_bpa_mesh_theory.ply", dec_mesh);
    # o3d.io.write_triangle_mesh(output_path+"S_MFPAD_37_bpa_mesh_theory.ply", dec_mesh);
    # o3d.io.write_triangle_mesh(output_path+"S_MFPAD_4_bpa_mesh_theory.ply", dec_mesh);
    o3d.io.write_triangle_mesh(output_path+"S_MFPAD_tot_p_mesh_theory.ply", poisson_mesh);
    # o3d.io.write_triangle_mesh(output_path+"S_MFPAD_37_p_mesh_theory.ply", poisson_mesh);
    # o3d.io.write_triangle_mesh(output_path+"S_MFPAD_4_p_mesh_theory.ply", poisson_mesh);
    # o3d.io.write_triangle_mesh(output_path+"S_MFPAD_p_mesh_c_theory.ply", p_mesh_crop);

# my_lods = lod_mesh_export(p_mesh_crop, [100000,50000,10000,1000,100], ".ply", output_path, enant, optimization)
my_lods = lod_mesh_export(poisson_mesh, [100000,50000,10000,1000,100], "_MFPAD_tot_th.ply", output_path, enant, optimization)
# my_lods = lod_mesh_export(poisson_mesh, [100000,50000,10000,1000,100], "_MFPAD_37_th.ply", output_path, enant, optimization)
# my_lods = lod_mesh_export(poisson_mesh, [100000,50000,10000,1000,100], "_MFPAD_4_th.ply", output_path, enant, optimization)
# my_lods = lod_mesh_export(poisson_mesh, [100000,50000,10000,1000,100], "_MFPAD_0_th.ply", output_path, enant, optimization)

In [ ]:
# o3d.visualization.draw_geometries([pcd, p_mesh_crop], mesh_show_back_face=True
# o3d.visualization.draw_geometries([pcd, bpa_mesh], mesh_show_back_face=True)
o3d.visualization.draw_geometries([pcd, poisson_mesh],mesh_show_back_face=True)
# o3d.visualization.draw_geometries([pcd, poisson_mesh[100000]],point_show_normal=True)

In [ ]:
# tri_mesh.show()
tri_mesh_pois.show()

In [ ]:
fig5 = go.Figure(go.Mesh3d(x=xgo, y=ygo, z=zgo, 
                           # alphahull=3.5, 
                           i=tigo, j=tjgo, k=tkgo, 
                           # i=ti, j=tj, k=tk, 
                           intensity=dgo,
                           colorscale="Viridis",
                           colorbar_len=0.75,
                           flatshading=True,
                           lighting=dict(ambient=0.5,
                                         diffuse=1,
                                         fresnel=4,        
                                         specular=0.5,
                                         roughness=0.05,
                                         facenormalsepsilon=0,
                                         vertexnormalsepsilon=0),
                          lightposition=dict(x=100,
                                             y=100,
                                            z=1000)))
fig5.update_layout(width=800, height=800)
fig5.show()
